<img src='https://raw.githubusercontent.com/dxkikuchi/SparkSnips/master/dsxbanner.jpg' width='75%'></img>

IBM Data Science Experience is an interactive, collaborative, cloud-based environment where data scientists can use multiple tools to activate their insights.  Data scientists can use the best of open source, tap into IBM's unique features, grow their capabilities, and share their successess.  In addition to all the features in the current preview, many new capabilities are being added including the ability to ingest Object Storage data with a single click, an enhanced user interface for version control, a facility to comment or chat about a notebook with others, and many more!

## New York State Restaurant Inspections Notebook
This notebook will provide insights from official restaurant inspections records for most of the state of New York and provide visualizations of that data.  The location of this data is defined below.  For additional details, please see <a href="https://health.data.ny.gov/Health/Food-Service-Establishment-Last-Inspection/cnih-y5dw" target="_blank">New York State Restaurant Ratings</a>

In [ ]:
nyr = 'https://health.data.ny.gov/api/views/cnih-y5dw/rows.csv?accessType=DOWNLOAD'

The csv (comma separated values) data will be read into a Pandas dataframe (nyr) and the first 5 records are displayed using the 'head()' method.<br>
Please attempt to write the code to read the csv data in the following cell without looking at the solution provided 2 cells from here.

In [ ]:
import pandas as pd
# Please add your code here
nyr.head()

<b>Solution</b>: Please copy and paste the following code into the previous cell where specified.<br>
nyr = pd.read_csv(nyr)<br>

Ingesting data can be as simple as using one line of code.  Similarly, many other sources of data can be ingested from Cloudant, DashDB, Object Storage, relational databases, and many others.

Note that the latitude and longitude are provided in the final column (Location1) of the retrieved data.  Functions are defined below to extract the latitude and longitude independently and will be used shortly.

In [ ]:
def get_lon(loc):
    if len(loc.split(",")) > 1:
        lon = loc.split(",")[1].replace(")", "").strip()
    else:
        lon = ""
    return lon

def get_lat(loc):
    if len(loc.split(",")) > 1:
        lat = loc.split(",")[0].replace("(", "").strip()
    else:
        lat = ""
    return lat

The data is transformed into a Spark dataframe (nyrDF) and a table is registered.  Spark dataframes are conceptually equivalent to a table in a relational database or a dataframe in R/Python, but with richer optimizations under the hood.  A table that is registered can be used in subsequent SQL statements.

In [ ]:
nyrDF = sqlContext.createDataFrame(nyr)
nyrDF.registerTempTable("nyrDF")

A Spark dataframe (nyvDF) will be created which contains a subset of the relevant columns and a table will be registered for violations.  Note that the previously defined functions are used to extract the latitude and longitude individually.<br>
Please attempt to write the code in the following cell to register the table 'nyvDF' without looking at the solution provided 2 cells from here.  You can review the previous cell to see how this was accomplished for 'nyrDF'.

In [ ]:
from pyspark.sql import Row
nyvDF = nyrDF.map(lambda x: Row(FACILITY=x.FACILITY, \
                                ADDRESS=x.ADDRESS, \
                                lat=get_lat(x.Location1), \
                                lon=get_lon(x.Location1), \
                                VIOLATIONS=x.VIOLATIONS, \
                                TOTAL_CRIT_VIOLATIONS=x["TOTAL # CRITICAL VIOLATIONS"])).toDF()
# Please add your code here

<b>Solution</b>: Please copy and paste the following code into the previous cell where specified.<br>
nyvDF.registerTempTable("nyvDF")

A SQL query is created that provides the restaurant name (facility), latitude, longitude and violations.  They are ordered by number of violations in descending order with the top 10 records displayed.

In [ ]:
query = """
select 
    FACILITY, 
    cast(lat as float) as lat,
    cast(lon as float) as lon,
    cast(TOTAL_CRIT_VIOLATIONS as int) as Violations
from nyvDF 
order by cast(TOTAL_CRIT_VIOLATIONS as decimal(10,2)) desc
limit 1000
"""
nyv1000 = sqlContext.sql(query)
nyv1000.show(10)

Brunel visualization will be used to map the latitude and longitude to a New York state map.  Colors represent the number of violations as described in the key.

In [ ]:
import brunel
nyv1000pan = nyv1000.toPandas()
%brunel map ('NY') + data('nyv1000pan') x(lon) y(lat) color(Violations) tooltip(FACILITY)

One of the many key strengths of Data Science Experience is the ability to easily search and quickly learn about various topics.  For example, to find articles, tutorials or notebooks on Brunel, click on the search button on the top right hand corner of this web page.  A side palette will appear where you can enter 'Brunel' or other topics of interest.

Pixiedust also provides charting and visualization.  It is an open source Python library that works as an add-on to Jupyter notebooks to improve the user experience of working with data.  For example, if you hover over the lonely yellow dot in the middle of New York State, you can see that it is for 'CAMP KINGSLEY - CC'.  By starting to type the value 'camp' in the 'Search table' text field below, the record will be displayed.  Numerous visualization are available with map support in the future.  In addition, the data can be downloaded as a file, or stashed to Cloudant or Object Storage.

In [ ]:
from pixiedust.display import *
display(nyv1000)